In [1]:
import xml.dom.minidom as Dom 
import os
import numpy as np
from PIL import Image

In [2]:
def diff_count(root_dir,img_dir,txt_dir):   
    diff_num=0;
    easy_num=0
    for root, dirs, files in os.walk(os.path.join(root_dir,img_dir)):
        #image path
        for file in files:
            image_name=file
            label_name=file[0:-4]+'.txt'          
            label_path=os.path.join(root_dir,txt_dir,label_name)
            label=open(label_path,'r')            
            first_line=True            
            for line in label.readlines():
                if first_line:
                    line=line.decode('utf-8-sig').strip()
                else:
                    line=line.strip()
                #print(line)
                bbs=line.split(',')                
                content=bbs[8]
                #print(content)
                content=''.join(ch for ch in content if ch.isalnum())
                content=content.upper()
                if content=='':
                    content='###'
                    difficult='1'
                    diff_num+=1
                else:
                    difficult='0'
                    easy_num+=1               
            label.close()   
    print("diff "+str(diff_num))
    print("easy "+str(easy_num))

In [3]:
def create_xml_rotated_icdar13_train(root_dir, save_dir, generate_file,img_dir,txt_dir):
    #root_dir
    generate_list=open(generate_file,'wt')
    #train_list=open('train.txt','wt')
    #test_list=open('test.txt','wt')
    count=0
    for root, dirs, files in os.walk(os.path.join(root_dir,img_dir)):
        #image path
        for file in files:
            image_name=file
            label_name=file[0:-4]+'.txt'
            image_path=os.path.join(root_dir,img_dir,image_name)
            sub_dir='train_images'
            #print(image_path)
            I=Image.open(image_path)
            image_width,image_height=I.size
            label_path=os.path.join(root_dir,txt_dir,label_name)
            label=open(label_path,'r')
            doc = Dom.Document() 
            root_node=doc.createElement('annotation')
            doc.appendChild(root_node)
            folder_node=doc.createElement('folder')
            folder_node_value=doc.createTextNode(sub_dir)
            folder_node.appendChild(folder_node_value)
            filename_node=doc.createElement('filename')
            filename_value=doc.createTextNode(image_name)
            filename_node.appendChild(filename_value)
            size_node=doc.createElement('size')
            root_node.appendChild(size_node)
            width_node=doc.createElement('width')
            width_value=doc.createTextNode(str(image_width))
            width_node.appendChild(width_value)
            height_node=doc.createElement('height')
            height_value=doc.createTextNode(str(image_height))
            height_node.appendChild(height_value)
            depth_node=doc.createElement('depth')
            depth_value=doc.createTextNode('3')
            depth_node.appendChild(depth_value)
            size_node.appendChild(width_node)
            size_node.appendChild(height_node)
            size_node.appendChild(depth_node)
            first_line=True
            count=0
            for line in label.readlines():
                if first_line:
                    line=line.decode('utf-8-sig').strip()
                else:
                    line=line.strip()
                #print(line)
                bbs=line.split(',')
                x1=int(float(bbs[0].encode("utf-8")))
                y1=int(float(bbs[1].encode("utf-8")))
                
                x2=int(float(bbs[2].encode("utf-8")))
                y2=int(float(bbs[3].encode("utf-8")))
                
                x3=int(float(bbs[4].encode("utf-8")))
                y3=int(float(bbs[5].encode("utf-8")))
                
                x4=int(float(bbs[6].encode("utf-8")))
                y4=int(float(bbs[7].encode("utf-8")))
                loc=np.array([x1 ,y1 ,x2 ,y2 ,x3, y3, x4, y4])
                loc_x=np.array([x1 ,x2 ,x3 ,x4])
                loc_y=np.array([y1 ,y2 ,y3 ,y4])
                
                xmin=(min(x1,x2,x3,x4))
                ymin=(min(y1,y2,y3,y4))
                xmax=(max(x1,x2,x3,x4))
                ymax=(max(y1,y2,y3,y4))
                if(xmax-xmin>=ymax-ymin):
                    #long aspect ratio
                    idx=np.argsort(loc_x)
                    if(loc_y[idx[0]]<=loc_y[idx[1]]):
                        x1=loc_x[idx[0]]
                        y1=loc_y[idx[0]]
                        x4=loc_x[idx[1]]
                        y4=loc_y[idx[1]]
                    else:
                        x1=loc_x[idx[1]]
                        y1=loc_y[idx[1]]
                        x4=loc_x[idx[0]]
                        y4=loc_y[idx[0]]
                    if(loc_y[idx[2]]<=loc_y[idx[3]]):
                        x2=loc_x[idx[2]]
                        y2=loc_y[idx[2]]
                        x3=loc_x[idx[3]]
                        y3=loc_y[idx[3]]
                    else:
                        x2=loc_x[idx[3]]
                        y2=loc_y[idx[3]]
                        x3=loc_x[idx[2]]
                        y3=loc_y[idx[2]]
                else:
                    #short aspect ratio
                    idx=np.argsort(loc_y)
                    if(loc_x[idx[0]]<=loc_x[idx[1]]):
                        x1=loc_x[idx[0]]
                        y1=loc_y[idx[0]]
                        x2=loc_x[idx[1]]
                        y2=loc_y[idx[1]]
                    else:
                        x1=loc_x[idx[1]]
                        y1=loc_y[idx[1]]
                        x2=loc_x[idx[0]]
                        y2=loc_y[idx[0]]
                    if(loc_x[idx[2]]<=loc_x[idx[3]]):
                        x4=loc_x[idx[2]]
                        y4=loc_y[idx[2]]
                        x3=loc_x[idx[3]]
                        y3=loc_y[idx[3]]
                    else:
                        x4=loc_x[idx[3]]
                        y4=loc_y[idx[3]]
                        x3=loc_x[idx[2]]
                        y3=loc_y[idx[2]]
                
                xmin=str(xmin)
                ymin=str(ymin)
                xmax=str(xmax)
                ymax=str(ymax)
                x1=str(x1)
                x2=str(x2)
                x3=str(x3)
                x4=str(x4)
                y1=str(y1)
                y2=str(y2)
                y3=str(y3)
                y4=str(y4)
                content=bbs[8]
                #print(content)
                content=''.join(ch for ch in content if ch.isalnum())
                content=content.upper()
                if content=='':
                    content='###'
                    difficult='0'
                else:
                    difficult='0'
                object_node=doc.createElement('object')
                root_node.appendChild(object_node)
                name_node=doc.createElement('name')
                name_value=doc.createTextNode('text')
                name_node.appendChild(name_value)
                content_node=doc.createElement('content')
                content_value=doc.createTextNode(content)
                content_node.appendChild(content_value)
                difficult_node=doc.createElement('difficult')
                difficult_value=doc.createTextNode(difficult)
                difficult_node.appendChild(difficult_value)
                bndbox_node=doc.createElement('bndbox')
                object_node.appendChild(bndbox_node)
                x1_node=doc.createElement('x1')
                x1_value=doc.createTextNode(x1)
                x1_node.appendChild(x1_value)
                y1_node=doc.createElement('y1')
                y1_value=doc.createTextNode(y1)
                y1_node.appendChild(y1_value)
                x2_node=doc.createElement('x2')
                x2_value=doc.createTextNode(x2)
                x2_node.appendChild(x2_value)
                y2_node=doc.createElement('y2')
                y2_value=doc.createTextNode(y2)
                y2_node.appendChild(y2_value)
                x3_node=doc.createElement('x3')
                x3_value=doc.createTextNode(x3)
                x3_node.appendChild(x3_value)
                y3_node=doc.createElement('y3')
                y3_value=doc.createTextNode(y3)
                y3_node.appendChild(y3_value)
                x4_node=doc.createElement('x4')
                x4_value=doc.createTextNode(x4)
                x4_node.appendChild(x4_value)
                y4_node=doc.createElement('y4')
                y4_value=doc.createTextNode(y4)
                y4_node.appendChild(y4_value)
                xmin_node=doc.createElement('xmin')
                xmin_value=doc.createTextNode(xmin)
                xmin_node.appendChild(xmin_value)
                ymin_node=doc.createElement('ymin')
                ymin_value=doc.createTextNode(ymin)
                ymin_node.appendChild(ymin_value)
                xmax_node=doc.createElement('xmax')
                xmax_value=doc.createTextNode(xmax)
                xmax_node.appendChild(xmax_value)
                ymax_node=doc.createElement('ymax')
                ymax_value=doc.createTextNode(ymax)
                ymax_node.appendChild(ymax_value)
                bndbox_node.appendChild(x1_node)
                bndbox_node.appendChild(y1_node)
                bndbox_node.appendChild(x2_node)
                bndbox_node.appendChild(y2_node)
                bndbox_node.appendChild(x3_node)
                bndbox_node.appendChild(y3_node)
                bndbox_node.appendChild(x4_node)
                bndbox_node.appendChild(y4_node)
                bndbox_node.appendChild(xmin_node)
                bndbox_node.appendChild(ymin_node)
                bndbox_node.appendChild(xmax_node)
                bndbox_node.appendChild(ymax_node)
                object_node.appendChild(difficult_node)
                object_node.appendChild(content_node)
                object_node.appendChild(name_node)
                object_node.appendChild(bndbox_node)
                count=count+1
            root_node.appendChild(folder_node)
            root_node.appendChild(filename_node)
            root_node.appendChild(size_node)
            for i in range(0,count): 
                root_node.appendChild(object_node)
            if count>0:
                xml_file_path=save_dir+image_name[0:len(image_name)-4]+'.xml'
                f = open(xml_file_path, 'w+') 
                f.write(doc.toprettyxml(indent = "\t", newl = "\n", encoding = "utf-8")) 
                f.close()
                generate_list.write(image_path+' '+xml_file_path+'\n')
                #train_list.write(image_path+'\n')
                #test_list.write(xml_file_path+'\n')

            label.close()
    generate_list.close()

In [8]:
#creat train data
root_dir=''
save_dir="xml_9000/"
generate_file="train.txt"
img_dir='image_9000/'
txt_dir='txt_9000/'
create_xml_rotated_icdar13_train(root_dir, save_dir, generate_file,img_dir,txt_dir)

In [5]:
#creat test data
root_dir=''
save_dir="xml_1000/"
generate_file="test.txt"
img_dir='image_1000/'
txt_dir='txt_1000/'
create_xml_rotated_icdar13_train(root_dir, save_dir, generate_file,img_dir,txt_dir)

In [6]:
img_dir='image_1000/'
txt_dir='txt_1000/'
diff_count(root_dir,img_dir,txt_dir)

diff 7384
easy 14794


In [7]:
img_dir='image_9000/'
txt_dir='txt_9000/'
diff_count(root_dir,img_dir,txt_dir)

diff 52961
easy 124725
